In [ ]:
import os
import dotenv

dotenv.load_dotenv(override=True)

# override input to mitigate jupyter's buffering

import builtins

def input(*args, **kwargs):
    print(*args, end='', flush=True)
    return builtins.input(*args, **kwargs)

input.__doc__ = builtins.input.__doc__

True

In [3]:
from langchain_groq import ChatGroq
from typing_extensions import TypedDict
from langchain.agents import tool, AgentExecutor
from langgraph.prebuilt import create_react_agent

model_name = 'qwen-qwq-32b'
llm = ChatGroq(
    model=model_name,
    temperature=0.4,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

In [5]:
class ProfileData(TypedDict):
    name: str
    age: int
    genre: str
    color: str
    city: str

In [6]:
profile_saved = False

@tool
def validate_name(name: str) -> str:
    '''Функция для проверки имени. Имя должно звучать правдоподобно.
    Это не должно быть нарицательным, названием географического объекта.
    Оно должно быть написано на русском языке в именительном падеже единственного числа.'''
    print("Вызываем тул для валидации имени")
    # игрушечная логика
    if name in ['Ярополк', 'Василий', 'Александр', 'Петр']:
        return 'Это корректное имя!'
    return 'Это не корректное имя!'

@tool
def validate_city(city: str) -> str:
    """Проверяет, существует ли город с таким названием.
    Если нет, то возвращает ошибку.
    Если есть, то возвращает сообщение об успешной проверке.
    """
    print("Вызываем тул для проверки существования города")
    # игрушечная логика
    cities = ["Москва", "Санкт-Петербург", "Казань", "Екатеринбург", "Новосибирск", "Владивосток"]
    if city not in cities:
        return f"❌ Ошибка: города с названием '{city}' не существует."
    return f"✅ Город '{city}' существует."

@tool
def save_profile(data: ProfileData) -> str:
    """Сохраняет профиль пользователя.
    Профиль пользователя состоит из следующих полей:
    - name: Имя
    - age: Возраст - целое число больше нуля. Если пользователь ввёл строку или не число, то это неверный ответ. Например, 2.5 или "-2" - это не возраст.
    - genre: Любимый жанр книг
    - city: Любимый город
    - color:Любимый цвет
    
    Возвращает сообщение об успешном сохранении профиля.
    """
    global profile_saved
    print("\n✅ Все данные собраны:")
    for k, v in data.items():
        print(f"- {k}: {v}")
    profile_saved = True
    return "Профиль успешно сохранён."

In [ ]:
main_prompt = """
Ты - охотник за информацией пользователя, и твоя следующая и самая важная задача - собрать его профиль.
Профиль состоит из следующих полей:
- name: Имя - должно быть корректным. Воспользуйся помощником для валидации имени.
- age: Возраст - целое число больше нуля. Если пользователь ввёл строку или не число, то это неверный ответ. Например, 2.5 или "-2" - это не возраст.
- genre: Любимый жанр книг
- city: Любимый город - город должен существовать. Воспользуйся помощником для проверки существования города.
- color:Любимый цвет

Для выяснения имени и любимого города воспользуйся помощниками.
Задавай пользователю вопросы и запрашивай у него данные о его профиле.

Когда все данные собраны, вызывай помощника для сохранения профиля.
"""

name_prompt = """
Ты - мастер по получению имени от пользователя. Задавай пользователю вопросы, пока не получишь правильное имя.
"""